In [83]:
import json, sys, random
import numpy as np
import pandas as pd

from PIL import Image, ImageDraw

from matplotlib import pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
from torchvision import transforms

from tqdm import tqdm_notebook
import seaborn as sns

In [3]:
f = open('shipsnet.json')
dataset = json.load(f)
f.close()

In [4]:
x_data = np.array(dataset['data']).astype('uint8')
y_data = np.array(dataset['labels']).astype('uint8')

In [5]:
x_data.shape

(4000, 19200)

In [6]:
x_train = x_data.reshape([-1, 3, 80, 80])

In [22]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_data, test_size=0.2)

In [87]:
from torchsample import TensorDataset

train_imgs = torch.from_numpy(x_train).float()
train_targets = torch.from_numpy(y_train).long()
train_dataset = TensorDataset(train_imgs, train_targets, 
                              input_transform=transforms.Compose([transforms.RandomHorizontalFlip(),
                                                                  transforms.RandomVerticalFlip()]))
val_imgs = torch.from_numpy(x_val).float()
val_targets = torch.from_numpy(y_val).long()
val_dataset = TensorDataset(val_imgs, val_targets)

In [55]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.batch = nn.BatchNorm2d(3)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(25600, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
        
    def forward(self, x):
        x = self.batch(x)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [80]:
net = Net()

In [81]:
epochs = 2
criterion = nn.CrossEntropyLoss()
optimizer = Adam(net.parameters())

In [64]:
class AverageMeter(object):
    
    def __init__(self, window_size=None):
        self.length = 0
        self.val = 0
        self.sum = 0
        self.count = 0
        self.window_size = window_size
        
    def reset(self):
        self.length = 0
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        
    def update(self, val, n=1):
        if self.window_size and (self.count >= self.window_size):
            self.reset()
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def accuracy(y_true, y_pred):
    y_true = y_true.float()
    _, y_pred = torch.max(y_pred, dim=-1)
    return (y_pred.float() == y_true).float().mean()

def fit(train, val, epochs, batch_size):
    print('train on {} images validate on {} images'.format(len(train), len(val)))
    net.train()
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val, batch_size=batch_size, shuffle=False)
    for epoch in tqdm_notebook(range(epochs), total=epochs):
        running_loss = AverageMeter()
        running_accuracy = AverageMeter()
        val_loss_meter = AverageMeter()
        val_acc_meter = AverageMeter()
        pbar = tqdm_notebook(train_loader, total=len(train_loader))
        for data, target in pbar:
            data, target = Variable(data), Variable(target)
            output = net(data)
            loss = criterion(output, target)
            acc = accuracy(target.data, output.data)
            running_loss.update(loss.data)
            running_accuracy.update(acc)
            pbar.set_description('[loss: {:.4f} | acc: {:.4f} ]'.format(running_loss.avg, running_accuracy.avg))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('[ loss: {:.4f} | acc: {:.4f} ]'.format(running_loss.avg, running_accuracy.avg))
        for val_data, val_target in val_loader:
            val_data, val_target = Variable(val_data), Variable(val_target)
            output = net(val_data)
            val_loss = criterion(output, val_target)
            val_acc = accuracy(val_target.data, output.data)
            val_loss_meter.update(val_loss.data)
            val_acc_meter.update(val_acc)
        pbar.set_description('[ loss: {:.4f} | acc:{:.4f} | vloss: {:.4f} | vacc: {:.4f} ]'.format(
            running_loss.avg, running_accuracy.avg, val_loss_meter.avg, val_acc_meter.avg))
        print('[ loss: {:.4f} | acc: {:.4f} | vloss: {:.4f} | vacc: {:.4f} ]'.format(
                running_loss.avg, running_accuracy.avg, val_loss_meter.avg, val_acc_meter.avg))

In [65]:
fit(train_dataset, val_dataset, 5, 100)

train on 3200 images validate on 800 images


[ loss: 0.0473 | acc: 0.9822 ]
[ loss: 0.0473 | acc: 0.9822 | vloss: 0.0606 | vacc: 0.9787 ]


[ loss: 0.0234 | acc: 0.9919 ]
[ loss: 0.0234 | acc: 0.9919 | vloss: 0.0530 | vacc: 0.9825 ]


[ loss: 0.0121 | acc: 0.9966 ]
[ loss: 0.0121 | acc: 0.9966 | vloss: 0.0527 | vacc: 0.9862 ]


[ loss: 0.0061 | acc: 0.9975 ]
[ loss: 0.0061 | acc: 0.9975 | vloss: 0.0478 | vacc: 0.9875 ]


KeyboardInterrupt: 

In [ ]:
fit(train_dataset, val_dataset, 5, 100)